In [2]:
# %%
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

train_data = pd.read_csv('train.csv')

In [3]:
features = ['LotArea', 'OverallQual', 'YearBuilt', '1stFlrSF', '2ndFlrSF']
X = train_data[features]
y = train_data['SalePrice']

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

house_model = RandomForestRegressor(n_estimators=100, random_state=42)

In [5]:
house_model.fit(X_train, y_train)

predictions = house_model.predict(X_val)

mae = mean_absolute_error(y_val, predictions)
r2 = r2_score(y_val, predictions)

print(f"Erro Médio Absoluto (MAE): ${mae:,.2f}")
print(f"R² Score (Poder de explicação): {r2:.4f}")

Erro Médio Absoluto (MAE): $20,708.30
R² Score (Poder de explicação): 0.8656
